In [1]:
import pandas as pd
import ib_insync as ib 
import nest_asyncio
from datetime import datetime
nest_asyncio.apply()

In [2]:
### Fonction qui te donne l'ensemble de tes pos de IBKR ?
### comment on trade en IBKR ? 
### Mettre en prod le simulated paper trading pour Crude Oil 

### -> tester nos deux backtest via crude oil (vs QuantConnect)
### -> 

In [ ]:
from TrendFollowCrudeOil import BaseSignal, CrudeOilTFSignal

def SimulateSignal(
        signal: BaseSignal, 
        date_range: list[str | datetime | pd.Timestamp] | pd.DatetimeIndex,
        ib_client: ib.IB
    ) -> pd.Series:

    assert ib_client.isConnected(), 'Failed to connect to IB'    
    return pd.Series({
        date: signal.get_value(date) 
        for date in date_range
    })

with ib.IB() as ib_client:
    ib_client.connect('127.0.0.1', 7497, clientId=1)
    assert ib_client.isConnected(), 'Failed to connect to IB'
    signal = CrudeOilTFSignal(ib_client)
    values = SimulateSignal(
        signal, 
        date_range=pd.date_range(start='2025-05-01', end=datetime.today()),
        ib_client=ib_client
    )

values

In [ ]:
from TrendFollowCrudeOil import BaseSignal
import time 

def get_current_position_on_contract(
        all_positions: list[ib.Position], 
        contract: ib.Future,
    ) -> int:

    for pos in all_positions:
        if (
            pos.contract.symbol == contract.symbol 
            and 
            pos.contract.lastTradeDateOrContractMonth == contract.lastTradeDateOrContractMonth
            ):
            return pos.position
    return 0

def place_order(
        qtity_to_buy: int, 
        contract: ib.Future, 
        ib_client: ib.IB,
    ) -> None:

    ### UNTESTED. I DON'T THINK IT WORKS.
    
    if qtity_to_buy == 0:
        return
    elif qtity_to_buy > 0:
        order = ib.MarketOrder('BUY', qtity_to_buy)
    elif qtity_to_buy < 0:
        order = ib.MarketOrder('SELL', -qtity_to_buy)
    print(f"Placing order for {qtity_to_buy} of {contract}")
    
    ib_client.placeOrder(contract, order)
    time.sleep(3)

def ExecuteSignal(signal: BaseSignal, ib_client: ib.IB, multiplier: int = 1) -> None:
        assert ib_client.isConnected(), 'Failed to connect to IB'

        today = pd.Timestamp.now()
        target_position = signal.get_value(today) * multiplier

        current_position = get_current_position_on_contract(
            ib_client.positions(), signal.contract
        )

        place_order(
            qtity_to_buy = int(target_position - current_position),
            contract = signal.contract,
            ib_client = ib_client
        )

        final_position = get_current_position_on_contract(
            ib_client.positions(), signal.contract, 
        )
        assert (
            final_position == int(target_position)
        ), f'Position not updated correctly. Expected: {target_position}, Got: {final_position}'


with ib.IB() as ib_client:
    ib_client.connect('127.0.0.1', 7497, clientId=1)
    assert ib_client.isConnected(), 'Failed to connect to IB'
    signal = CrudeOilTFSignal(ib_client)
    value = ExecuteSignal(signal, ib_client, multiplier=10)
    print(ib_client.positions())

value 